# Solving RBC Model with Dolo.jl

This worksheet demonstrates how to solve the RBC model with the [dolo](http://econforge.github.io/dolo/) library 
and how to generates impulse responses and stochastic simulations from the solution.

- This notebook is modified from `rbc_model` example distributed with dolo in : ``examples\notebooks\``.
- The model file(.yaml file) is in : ``examples\global_models\``


# YAML file for model specification

Dolo uses [YAML](http://www.yaml.org/spec/1.2/spec.html#Introduction) file which we can read locally or pull off the web. 

The YAML file works similarly to `.mod` file in Dynare, which does a configuration of the model. It includes the following:

1. In `symbols` section, specify different kinds of variables in the model:
    - `parameters:` deep parameters of the model, exp: $\beta$
    - `exogenous:` exogenous shocks of the model, exp: $\epsilon$ (iid shock on productivity)
    - `states:` state variables of the model, exp: $k$ (capital stock)
    - `controls:` control/choice variables of the model, exp: $n$ (labor)
    - `values:` value function in terms of Bellman Equation; `rewards:` utility function.

2. "auxiliary variables": these are variables that can be expressed as a closed form of the formerly-defined variables. Defining these variables will help to simplify the system of equations and sometimes these variables have particular economic meaning. These variables are defined in `definitions`. Once defined, one could use them in model equations.

3. Model section, specified using command `equations:`, consist of:

   Essential parts:
    
    - `arbitrage:` Euler Equation, notice the EE specification is usually accompanied by complementarity condition included in the `arbitrage` configuration.

    - `transition:` Law of motion, this should include both things like capital accumulation and process of shocks.

    Aditionals:

    - `expectations:` A function of purely expected future values.

    - `felicity:` defining utility function; `values:` defining value function;

    - `direct response:` additional equations that may speed up computation.


For more options in model specification please refer to http://www.econforge.org/Dolo.jl/latest/model_specification.html

# Example of RBC

In [1]:
using Dolo
filename="rbc_dtcc_iid.yaml";

In [2]:
;cat $filename

name: Real Business Cycle

model_type: dtcc

symbols:

   exogenous: [e_z]
   states: [z, k]
   controls: [n, i]
   expectations: [m]
   values: [V]
   parameters: [beta, sigma, eta, chi, delta, alpha, rho, zbar, sig_z]
   rewards: [u]

definitions:
    y: exp(z)*k^alpha*n^(1-alpha)
    c: y - i
    rk: alpha*y/k
    w: (1-alpha)*y/n

equations:

    arbitrage:
        - chi*n^eta*c^sigma - w                      | 0.0 <= n <= inf
        - 1 - beta*(c/c(1))^(sigma)*(1-delta+rk(1))  | 0.0 <= i <= inf


    transition:
        - z = rho*z(-1) + e_z
        - k = (1-delta)*k(-1) + i(-1)

    value:
        - V = c^(1-sigma)/(1-sigma) - chi*n^(1+eta)/(1+eta) + beta*V(1)

    felicity:
        - u =  c^(1-sigma)/(1-sigma) - chi*n^(1+eta)/(1+eta)

    expectation:
        - m = beta/c(1)^sigma*(1-delta+rk(1))

    direct_response:
        - n = ((1-alpha)*exp(z)*k^alpha*m/chi)^(1/(eta+alpha))
        - i = exp(z)*k^alpha*n^(1-alpha) - (m)^(-1/sigma)

calibration:

    # parameters
    beta 

`yaml_import(filename)` reads the YAML file and generates a model object. 

In [3]:
model = yaml_import(filename)

name,Real Business Cycle
filename,rbc_dtcc_iid.yaml
Type,Equation
expectation,\[m_{t} = \frac{\beta}{\left(c_{t+1}\right)^{\sigma}} \left(1-\delta\right)+rk_{t+1}\]
value,\[V_{t} = \left(\frac{\left(c_{t}\right)^{\left(1-\sigma\right)}}{\left(1-\sigma\right)}-\frac{\chi \left(n_{t}\right)^{1+\eta}}{1+\eta}\right)+\beta V_{t+1}\]
transition,"\[z_{t} = \rho z_{t-1}+e_{z,t}\]"
,\[k_{t} = \left(1-\delta\right) k_{t-1}+i_{t-1}\]
direct_response,\[n_{t} = \left(\frac{\left(1-\alpha\right) \text{exp}\left(z_{t}\right) \left(k_{t}\right)^{\alpha} m_{t}}{\chi}\right)^{\frac{1}{\eta+\alpha}}\]
,\[i_{t} = \left(\text{exp}\left(z_{t}\right) \left(k_{t}\right)^{\alpha} \left(n_{t}\right)^{\left(1-\alpha\right)}-\left(m_{t}\right)^{\frac{-1}{\sigma}}\right)\]
felicity,\[u_{t} = \left(\frac{\left(c_{t}\right)^{\left(1-\sigma\right)}}{\left(1-\sigma\right)}-\frac{\chi \left(n_{t}\right)^{1+\eta}}{1+\eta}\right)\]
arbitrage,\[\left(\chi \left(n_{t}\right)^{\eta} \left(c_{t}\right)^{\sigma}-w_{t}\right)\]


# Steady State values

The model file already has values for steady-state variables stated in the calibration section so we can go ahead and check that they are correct by computing the model equations at the steady state.

In [4]:
model.calibration[:controls]

2-element Array{Float64,1}:
 0.33    
 0.233874

# Solving the model

Dolo offers several [algorithms](http://www.econforge.org/Dolo.jl/latest/algos.html) to solve the model. One is `time_iteration`, and its alternative faster command `improved_time_iteration`.

In [5]:
@time sol_global = improved_time_iteration(model)

------------------------------------------------------------------------------------------------------------------------
N	f_x		d_x	Time_residuals	Time_inversion	Time_search	Lambda_0	N_invert	N_search	
------------------------------------------------------------------------------------------------------------------------
1      1.188337e+00 2.858994e-01     4.6152         2.1378         0.17598        0.000           60               3    
2      8.783762e-01 7.686131e-02     0.0056         0.0265         0.00116        0.000           72               1    
3      1.045269e-01 0.000000e+00     0.0051         0.0469         0.00195        0.000           136              1    
4      2.948941e-02 2.725911e-03     0.0059         0.0251         0.00104        0.000           98               1    
5      9.991518e-04 0.000000e+00     0.0111         0.0186         0.00115        0.000           98               1    
6      1.702487e-06 9.957221e-08     0.0057         0.0300         0.001

Results of Improved Time Iteration Algorithm
 * Number of iterations: 7
 * Complementarities: true
 * Decision Rule type: Dolo.CubicDR{Dolo.EmptyGrid,Dolo.CartesianGrid{2},2,2}
 * Convergence: true
 * Contractivity: 0.0
   * |x - x'| < 1.0e-08: true


# Decision rule

... and we get the decision rule and plot optimal investment and labour for different levels of capital.

In [6]:
dr_global = sol_global.dr
tab_global = Dolo.tabulate(model, dr_global, :k)

2-dimensional AxisArray{Float64,2,...} with axes:
    :V, Symbol[:e_z, :z, :k, :n, :i, :w, :rk, :y, :c]
    :k, [8.41948, 8.43838, 8.45728, 8.47618, 8.49508, 8.51398, 8.53287, 8.55177, 8.57067, 8.58957  …  10.1204, 10.1393, 10.1582, 10.1771, 10.196, 10.2149, 10.2338, 10.2527, 10.2716, 10.2905]
And data, a 9×100 Array{Float64,2}:
 0.0        0.0        0.0       …   0.0         0.0         0.0      
 0.0        0.0        0.0           0.0         0.0         0.0      
 8.41948    8.43838    8.45728      10.2527     10.2716     10.2905   
 0.345165   0.344837   0.34451       0.316939    0.316681    0.316423 
 0.243497   0.243299   0.2431        0.224154    0.223956    0.223757 
 1.92252    1.92454    1.92657   …   2.11023     2.11208     2.11393  
 0.0388196  0.0387366  0.038654      0.0321297   0.0320726   0.0320156
 0.990426   0.990529   0.99063       0.998228    0.998291    0.998352 
 0.746929   0.74723    0.74753       0.774074    0.774335    0.774595 

In [7]:
using Plots
gr()

figure_1 = plot([tab_global[:k] tab_global[:k]], [tab_global[:i] tab_global[:n]], layout = 2, label = ["Global" "Global"], title = ["Investment" "Labour"], ylabel = ["i" "n"], line = 3)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 8.5 
 
 
 9.0 
 
 
 9.5 
 
 
 10.0 
 
 
 0.225 
 
 
 0.230 
 
 
 0.235 
 
 
 0.240 
 
 
 Investment 
 
 
 i 
 
<polyline clip-path="url(#clip8602)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 61.1494,31.4961 63.3632,34.9543 65.577,38.4178 67.7908,41.8917 70.0046,45.3811 72.2184,48.8913 74.4322,52.4262 76.646,55.9805 78.8597,59.5445 81.0735,63.1084 
 83.2873,66.6625 85.5011,70.1977 87.7149,73.7134 89.9287,77.2162 92.1425,80.7127 94.3563,84.2097 96.5701,87.7137 98.7839,91.2275 100.998,94.7477 103.211,98.2709 
 105.425,101.794 107.639,105.312 109.853,108.824 112.067,112.331 114.28,115.834 116.494,119.336 118.708,122.837 120.922,126.34 123.136,129.844 125.349,133.348 
 127.563,136.852 129.777,140.355 131.991,143.856 134.205,147.354 136.418,150.851 138.632,154.347 140.846,157.842 143.06,161.336 145.274,164.829 147.487,168.322 
 149.701,171.813 151.915,175.304 154.129,178.794 156.343,182.282 158.556,185.769 160.77,189.255 162.984,192.74 165.198,196.224 167.411,199.706 169.625,203.188 
 171.839,206.669 174.053,210.148 176.267,213.627 178.48,217.105 180.694,220.581 182.908,224.056 185.122,227.53 187.336,231.003 189.549,234.475 191.763,237.945 
 193.977,241.414 196.191,244.883 198.405,248.35 200.618,251.816 202.832,255.282 205.046,258.746 207.26,262.21 209.474,265.671 211.687,269.13 213.901,272.588 
 216.115,276.044 218.329,279.5 220.543,282.955 222.756,286.41 224.97,289.866 227.184,293.321 229.398,296.775 231.612,300.225 233.825,303.672 236.039,307.113 
 238.253,310.551 240.467,313.988 242.68,317.426 244.894,320.868 247.108,324.316 249.322,327.767 251.536,331.218 253.749,334.664 255.963,338.101 258.177,341.525 
 260.391,344.935 262.605,348.337 264.818,351.738 267.032,355.144 269.246,358.561 271.46,361.993 273.674,365.438 275.887,368.891 278.101,372.351 280.315,375.813 
 
 "/>
 
 
 
 
 Global 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 8.5 
 
 
 9.0 
 
 
 9.5 
 
 
 10.0 
 
 
 0.320 
 
 
 0.325 
 
 
 0.330 
 
 
 0.335 
 
 
 0.340 
 
 
 0.345 
 
 
 Labour 
 
 
 n 
 
<polyline clip-path="url(#clip8603)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 361.149,31.4961 363.363,35.4175 365.577,39.3377 367.791,43.2556 370.005,47.1698 372.218,51.0791 374.432,54.9824 376.646,58.8779 378.86,62.7637 381.074,66.6381 
 383.287,70.4989 385.501,74.3447 387.715,78.1753 389.929,81.9926 392.143,85.7981 394.356,89.5935 396.57,93.3806 398.784,97.16 400.998,100.931 403.211,104.692 
 405.425,108.444 407.639,112.184 409.853,115.913 412.067,119.63 414.28,123.337 416.494,127.034 418.708,130.72 420.922,134.398 423.136,138.066 425.349,141.725 
 427.563,145.374 429.777,149.013 431.991,152.642 434.205,156.262 436.418,159.871 438.632,163.471 440.846,167.062 443.06,170.643 445.274,174.214 447.487,177.777 
 449.701,181.33 451.915,184.874 454.129,188.408 456.343,191.934 458.556,195.45 460.77,198.956 462.984,202.454 465.198,205.943 467.411,209.423 469.625,212.893 
 471.839,216.355 474.053,219.808 476.267,223.252 478.48,226.688 480.694,230.114 482.908,233.532 485.122,236.941 487.336,240.341 489.549,243.733 491.763,247.116 
 493.977,250.491 496.191,253.857 498.405,257.215 500.618,260.564 502.832,263.905 505.046,267.238 507.26,270.562 509.474,273.878 511.687,277.185 513.901,280.484 
 516.115,283.775 518.329,287.058 520.543,290.332 522.756,293.599 524.97,296.859 527.184,300.11 529.398,303.354 531.612,306.589 533.825,309.816 536.039,313.033 
 538.253,316.243 540.467,319.444 542.68,322.638 544.894,325.826 547.108,329.007 549.322,332.183 551.536,335.35 553.749,338.51 555.963,341.66 558.177,344.8 
 560.391,347.929 562.605,351.05 564.818,354.162 567.032,357.268 569.246,360.367 571.46,363.462 573.674,366.553 575.887,369.642 578.101,372.728 580.315,375.813 
 
 "/>
 
 
 
 
 Global

Dolo also offers a convenient way to change parameter values in configuration. Use `set_calibration!(model,para=val)` command one can replace the original parameter value in the model file.

We can use this feature to do comparative stat easily. For example, let's consider a change in the value of $\delta$ in the model:

In [8]:
original_delta=model.calibration.flat[:delta] 

drs = []
delta_values = linspace(0.02, 0.05,5)
for val in delta_values
    print(val)
    set_calibration!(model, delta=val)
    sol = time_iteration(model, verbose=false)
    push!(drs, sol.dr)
end

0.020.02750.0350.04250.05

In [9]:
figure_2 = plot()
for (i,dr) in enumerate(drs)
     sim = Dolo.tabulate(model, dr,:k)
     dv = delta_values[i]
     plot!(sim[:k],sim[:i], label="\\delta=$dv", ylabel = "i", title = "Investment", xlim = [3, 15], ylim = [0.19, 0.26], line = 3)
end

set_calibration!(model,delta=original_delta)

figure_2

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3 
 
 
 6 
 
 
 9 
 
 
 12 
 
 
 15 
 
 
 0.19 
 
 
 0.20 
 
 
 0.21 
 
 
 0.22 
 
 
 0.23 
 
 
 0.24 
 
 
 0.25 
 
 
 0.26 
 
 
 Investment 
 
 
 i 
 
<polyline clip-path="url(#clip8802)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 387.099,93.3637 388.14,94.5796 389.182,95.7973 390.223,97.0185 391.265,98.2448 392.306,99.478 393.348,100.719 394.389,101.967 395.431,103.218 396.472,104.469 
 397.514,105.716 398.555,106.956 399.597,108.19 400.638,109.419 401.68,110.646 402.721,111.873 403.763,113.103 404.804,114.336 405.846,115.571 406.887,116.807 
 407.929,118.042 408.97,119.276 410.011,120.508 411.053,121.738 412.094,122.966 413.136,124.193 414.177,125.421 415.219,126.649 416.26,127.877 417.302,129.106 
 418.343,130.334 419.385,131.561 420.426,132.788 421.468,134.014 422.509,135.239 423.551,136.464 424.592,137.688 425.634,138.912 426.675,140.135 427.717,141.359 
 428.758,142.582 429.8,143.804 430.841,145.026 431.882,146.247 432.924,147.468 433.965,148.688 435.007,149.908 436.048,151.128 437.09,152.347 438.131,153.565 
 439.173,154.783 440.214,156.001 441.256,157.218 442.297,158.435 443.339,159.651 444.38,160.867 445.422,162.082 446.463,163.297 447.505,164.511 448.546,165.725 
 449.588,166.938 450.629,168.151 451.671,169.364 452.712,170.576 453.753,171.787 454.795,172.999 455.836,174.209 456.878,175.419 457.919,176.629 458.961,177.837 
 460.002,179.045 461.044,180.253 462.085,181.46 463.127,182.668 464.168,183.875 465.21,185.083 466.251,186.29 467.293,187.495 468.334,188.7 469.376,189.902 
 470.417,191.103 471.459,192.303 472.5,193.504 473.542,194.706 474.583,195.91 475.625,197.116 476.666,198.321 477.707,199.525 478.749,200.725 479.79,201.921 
 480.832,203.112 481.873,204.301 482.915,205.489 483.956,206.678 484.998,207.871 486.039,209.069 487.081,210.271 488.122,211.476 489.164,212.683 490.205,213.891 
 
 "/>
<polyline clip-path="url(#clip8802)" style="stroke:#e26f46; stroke-width:3; stroke-opacity:1; fill:none" points="
 256.003,124.193 256.751,125.08 257.498,125.97 258.245,126.861 258.992,127.758 259.739,128.659 260.487,129.567 261.234,130.48 261.981,131.396 262.728,132.312 
 263.475,133.226 264.223,134.135 264.97,135.039 265.717,135.94 266.464,136.839 267.211,137.738 267.959,138.64 268.706,139.543 269.453,140.449 270.2,141.355 
 270.948,142.262 271.695,143.167 272.442,144.071 273.189,144.974 273.936,145.875 274.684,146.777 275.431,147.678 276.178,148.58 276.925,149.483 277.672,150.385 
 278.42,151.287 279.167,152.19 279.914,153.091 280.661,153.993 281.408,154.894 282.156,155.794 282.903,156.695 283.65,157.595 284.397,158.496 285.145,159.396 
 285.892,160.296 286.639,161.196 287.386,162.096 288.133,162.995 288.881,163.894 289.628,164.793 290.375,165.692 291.122,166.591 291.869,167.489 292.617,168.388 
 293.364,169.286 294.111,170.183 294.858,171.081 295.605,171.979 296.353,172.876 297.1,173.773 297.847,174.67 298.594,175.566 299.342,176.462 300.089,177.359 
 300.836,178.254 301.583,179.15 302.33,180.046 303.078,180.941 303.825,181.836 304.572,182.731 305.319,183.626 306.066,184.52 306.814,185.414 307.561,186.307 
 308.308,187.201 309.055,188.094 309.802,188.987 310.55,189.88 311.297,190.773 312.044,191.666 312.791,192.559 313.539,193.452 314.286,194.343 315.033,195.233 
 315.78,196.122 316.527,197.011 317.275,197.9 318.022,198.791 318.769,199.683 319.516,200.576 320.263,201.468 321.011,202.36 321.758,203.249 322.505,204.135 
 323.252,205.018 323.999,205.898 324.747,206.779 325.494,207.66 326.241,208.545 326.988,209.434 327.736,210.326 328.483,211.22 329.23,212.116 329.977,213.013 
 
 "/>
<polyline clip-path="url(#clip8802)" style="stroke:#3da44d; stroke-width:3; stroke-opacity:1; fill:none" points="
 176.867,167.775 177.437,168.458 178.007,169.142 178.576,169.828 179.146,170.518 179.715,171.212 180.285,171.912 180.855,172.615 181.424,173.321 181.994,174.027 
 1

We find that more durable capital leads to higher steady state investment and slows the rate of convergence for capital (the slopes are roughly the same, which implies that relative to steady state capital investment responds stronger at higher $\delta$, this in addition to the direct effect of depreciation).

# Use the model to simulate

We will use the deterministic steady-state as a starting point.

In [10]:
set_calibration!(model, delta=0.05);

## Impulse response functions

Let us plot the response of consumption and investment to a shock on productivity (to innovation `e_z`)

In [11]:
dr_global = time_iteration(model).dr
irf = response(model, dr_global, :e_z, 0.01)

------------------------------------------------------------------
It    ϵₙ              ηₙ=|xₙ-xₙ₋₁|    λₙ=ηₙ/ηₙ₋₁      Time            Newton steps
------------------------------------------------------------------
1     9.24e-01        1.28e-01        NaN             4.56e-02        7    
2     1.62e-01        4.14e-02        3.25e-01        1.97e-02        5    
3     8.99e-02        2.86e-02        6.90e-01        2.64e-02        5    
4     5.62e-02        2.03e-02        7.09e-01        2.64e-02        5    
5     3.79e-02        1.47e-02        7.27e-01        2.84e-02        4    
6     2.68e-02        1.10e-02        7.43e-01        1.47e-02        4    
7     1.96e-02        8.30e-03        7.57e-01        1.92e-02        4    
8     1.47e-02        6.39e-03        7.70e-01        1.52e-02        4    
9     1.13e-02        4.99e-03        7.81e-01        1.57e-02        4    
10    8.80e-03        3.95e-03        7.91e-01        2.45e-02        4    
11    6.96e-03        3

2-dimensional AxisArray{Float64,2,...} with axes:
    :V, Symbol[:e_z, :z, :k, :n, :i, :w, :rk, :y, :c]
    :T, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  31, 32, 33, 34, 35, 36, 37, 38, 39, 40]
And data, a 9×40 Array{Float64,2}:
 0.0        0.01       0.0        …  0.0         0.0         0.0       
 0.0        0.01       0.008         3.24519e-6  2.59615e-6  2.07692e-6
 4.19221    4.19238    4.20096       4.20105     4.20069     4.20035   
 0.330089   0.330999   0.330549      0.329813    0.329824    0.329834  
 0.209787   0.218194   0.216413      0.209692    0.209695    0.209699  
 1.55       1.56418    1.56281    …  1.55152     1.55145     1.55139   
 0.0601118  0.0608264  0.0605666     0.0599936   0.0599984   0.0600029 
 0.76364    0.77275    0.771023      0.763745    0.76374     0.763736  
 0.553853   0.554556   0.55461       0.554053    0.554045    0.554037  

The easiest way to plot IRF is simply using the stored `irf` values.

In [13]:
figure_3 = plot([irf[:z], irf[:i], irf[:n], irf[:c]], layout = 4, title = ["Productivity" "Investment" "Labour" "Consumption"], line = 2, legend=:none)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 0.000 
 
 
 0.002 
 
 
 0.004 
 
 
 0.006 
 
 
 0.008 
 
 
 0.010 
 
 
 Productivity 
 
<polyline clip-path="url(#clip9002)" style="stroke:#009af9; stroke-width:2; stroke-opacity:1; fill:none" points="
 58.5806,175.813 64.3506,31.4961 70.1206,60.3595 75.8906,83.4503 81.6605,101.923 87.4305,116.701 93.2005,128.524 98.9705,137.982 104.74,145.548 110.51,151.601 
 116.28,156.443 122.05,160.317 127.82,163.417 133.59,165.896 139.36,167.88 145.13,169.466 150.9,170.736 156.67,171.751 162.44,172.564 168.21,173.214 
 173.98,173.734 179.75,174.15 185.52,174.482 191.29,174.749 197.06,174.962 202.83,175.132 208.6,175.268 214.37,175.377 220.14,175.465 225.91,175.534 
 231.68,175.59 237.45,175.635 243.22,175.671 248.99,175.699 254.76,175.722 260.53,175.74 266.3,175.755 272.07,175.767 277.84,175.776 283.61,175.783 
 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 0.210 
 
 
 0.212 
 
 
 0.214 
 
 
 0.216 
 
 
 0.218 
 
 
 Investment 
 
<polyline clip-path="url(#clip9003)" style="stroke:#009af9; stroke-width:2; stroke-opacity:1; fill:none" points="
 355.286,173.613 361.056,31.4961 366.826,61.599 372.596,85.5677 378.366,104.642 384.136,119.81 389.906,131.861 395.676,141.426 401.446,149.008 407.216,155.008 
 412.986,159.748 418.756,163.483 424.526,166.419 430.296,168.718 436.066,170.511 441.835,171.902 447.605,172.974 453.375,173.793 459.145,174.413 464.915,174.876 
 470.685,175.214 476.455,175.456 482.225,175.621 487.995,175.727 493.765,175.788 499.535,175.813 505.305,175.812 511.075,175.791 516.845,175.755 522.615,175.709 
 528.385,175.655 534.155,175.596 539.925,175.535 545.695,175.471 551.465,175.408 557.235,175.345 563.005,175.283 568.775,175.222 574.545,175.164 580.315,175.108 
 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 0.32950 
 
 
 0.32975 
 
 
 0.33000 
 
 
 0.33025 
 
 
 0.33050 
 
 
 0.33075 
 
 
 Labour 
 
<polyline clip-path="url(#clip9004)" style="stroke:#009af9; stroke-width:2; stroke-opacity:1; fill:none" points="
 58.5806,310.698 64.3506,231.496 70.1206,270.641 75.8906,300.613 81.6605,323.338 87.4305,340.347 93.2005,352.858 98.9705,361.839 104.74,368.059 110.51,372.129 
 116.28,374.535 122.05,375.662 127.82,375.813 133.59,375.23 139.36,374.102 145.13,372.578 150.9,370.775 156.67,368.784 162.44,366.676 168.21,364.507 
 173.98,362.316 179.75,360.138 185.52,357.994 191.29,355.903 197.06,353.877 202.83,351.924 208.6,350.05 214.37,348.258 220.14,346.55 225.91,344.925 
 231.68,343.383 237.45,341.922 243.22,340.539 248.99,339.232 254.76,337.998 260.53,336.834 266.3,335.737 272.07,334.703 277.84,333.73 283.61,332.813 
 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 0.5540 
 
 
 0.5542 
 
 
 0.5544 
 
 
 0.5546 
 
 
 Consumption 
 
<polyline clip-path="url(#clip9005)" style="stroke:#009af9; stroke-width:2; stroke-opacity:1; fill:none" points="
 355.286,375.813 361.056,249.891 366.826,240.289 372.596,234.596 378.366,231.904 384.136,231.496 389.906,232.807 395.676,235.393 401.446,238.904 407.216,243.068 
 412.986,247.67 418.756,252.547 424.526,257.571 430.296,262.643 436.066,267.691 441.835,272.658 447.605,277.504 453.375,282.201 459.145,286.728 464.915,291.072 
 470.685,295.226 476.455,299.187 482.225,302.954 487.995,306.53 493.765,309.918 499.535,313.125 505.305,316.155 511.075,319.016 516.845,321.716 522.615,324.26 
 528.385,326.658 534.155,328.916 539.925,331.041 545.695,333.041 551.465,334.922 557.235,336.69 563.005,338.354 568.775,339.917 574.545,341.386 580.315,342.767 
 
 "/>

## Stochastic simulations

Now we run 1000 random simulations the result is an AxisArrays indexed by:
- $T$ the number of dates
- $N$ the number of simulations
- $V$ is the number of variables
(the actual ordering of the dimensions is irrelevent if one uses the AxisArrays indexing routines)

In [14]:
sim = simulate(model, dr_global, N=1000, T=500)
# actual ordering of the data: (N,V,T)
size(sim.data)

(1000, 9, 500)

In [27]:
using AxisArrays
# Extract simulated data with all samples for specific factors over time
sim_z = sim[Axis{:V}(:z)].data # An array with dimension 1000*500
sim_k = sim[Axis{:V}(:k)].data
sim_n = sim[Axis{:V}(:n)].data
sim_i = sim[Axis{:V}(:i)].data
sim_c = sim[Axis{:V}(:c)].data;

We plot the responses of consumption, investment and labour to the stochastic path of productivity.

In [30]:
# May have trouble increasing the number plotted on OSX.
imin = 500
imax = 550
figure_4 = plot(layout = 4)
for i in imin:imax
    figure_4 = plot!([sim_z[i, :], sim_i[i, :], sim_n[i, :], sim_c[i, :]], layout = 4, color=:red, alpha = 0.1, title = ["Productivity" "Investment" "Labour" "Consumption"], xlim = [-10, 510], ylim = [[-0.12, 0.12] [0.12, 0.32] [0.31, 0.355] [0.53, 0.575]], legend=:none)
end
figure_4

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 500 
 
 
 -0.10 
 
 
 -0.05 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 Productivity 
 
<polyline clip-path="url(#clip9602)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:0.1; fill:none" points="
 50.7687,103.655 51.2265,109.27 51.6843,115.315 52.1421,127.39 52.5998,111.71 53.0576,106.066 53.5154,111.253 53.9731,115.854 54.4309,114.015 54.8887,109.031 
 55.3465,117.257 55.8042,112.953 56.262,114.188 56.7198,105.195 57.1776,86.2029 57.6353,80.3559 58.0931,95.6938 58.5509,108.508 59.0087,120.677 59.4664,119.217 
 59.9242,115.328 60.382,111.858 60.8398,94.752 61.2975,100.531 61.7553,103.084 62.2131,104.434 62.6709,103.479 63.1286,93.4931 63.5864,89.6388 64.0442,89.4897 
 64.502,93.6917 64.9597,106.461 65.4175,94.51 65.8753,101.601 66.333,106.226 66.7908,106.999 67.2486,98.7611 67.7064,102.824 68.1641,90.1433 68.6219,102.323 
 69.0797,91.6486 69.5375,97.7176 69.9952,93.555 70.453,102.301 70.9108,105.179 71.3686,106.071 71.8263,96.1649 72.2841,94.7457 72.7419,87.0525 73.1997,74.9661 
 73.6574,81.7809 74.1152,84.5485 74.573,86.3201 75.0308,97.635 75.4885,105.812 75.9463,93.1498 76.4041,85.0171 76.8618,101.498 77.3196,104.335 77.7774,117.419 
 78.2352,109.399 78.6929,86.489 79.1507,84.7472 79.6085,93.0005 80.0663,99.3965 80.524,102.574 80.9818,101.013 81.4396,96.8147 81.8974,97.7166 82.3551,104.769 
 82.8129,123.671 83.2707,142.956 83.7285,139.753 84.1862,141.108 84.644,122.058 85.1018,122.714 85.5596,102.27 86.0173,102.108 86.4751,78.8957 86.9329,69.5839 
 87.3907,72.0369 87.8484,66.1922 88.3062,59.5287 88.764,57.7034 89.2217,58.0305 89.6795,47.9359 90.1373,61.5199 90.5951,57.7166 91.0528,53.1067 91.5106,70.6742 
 91.9684,87.1098 92.4262,75.7806 92.8839,89.1372 93.3417,77.5552 93.7995,67.7115 94.2573,79.1027 94.715,95.0764 95.1728,99.2734 95.6306,110.975 96.0884,104.71 
 96.5461,101.914 97.0039,96.506 97.4617,109.872 97.9195,112.657 98.3772,116.471 98.835,114.21 99.2928,103.083 99.7505,100.896 100.208,125.144 100.666,121.043 
 101.124,115.015 101.582,118.761 102.039,117.858 102.497,118.912 102.955,120.328 103.413,113.046 103.871,105.723 104.328,104.054 104.786,102.588 105.244,112.274 
 105.702,111.706 106.159,108.214 106.617,112.145 107.075,116.224 107.533,120.186 107.99,113.305 108.448,106.654 108.906,110.28 109.364,108.722 109.822,109.689 
 110.279,111.82 110.737,96.6299 111.195,107.157 111.653,101.736 112.11,106.298 112.568,103.834 113.026,98.8345 113.484,107.378 113.942,112.043 114.399,101.289 
 114.857,121.352 115.315,132.529 115.773,122.76 116.23,115.13 116.688,105.642 117.146,101.571 117.604,103.271 118.062,100.324 118.519,106.083 118.977,111.783 
 119.435,101.958 119.893,103.387 120.35,92.56 120.808,81.329 121.266,95.595 121.724,105.819 122.181,107.905 122.639,95.3255 123.097,105.919 123.555,112.212 
 124.013,95.5056 124.47,109.913 124.928,126.272 125.386,121.954 125.844,138.01 126.301,138.274 126.759,130.756 127.217,123.716 127.675,116.867 128.133,124.256 
 128.59,128.143 129.048,135.941 129.506,115.95 129.964,112.375 130.421,110.719 130.879,106.576 131.337,105.032 131.795,102.166 132.253,105.478 132.71,106.604 
 133.168,109.305 133.626,110.981 134.084,122.603 134.541,119.441 134.999,120.662 135.457,114.647 135.915,96.5993 136.372,99.6994 136.83,91.3475 137.288,89.3859 
 137.746,91.4103 138.204,104.964 138.661,106.707 139.119,102.747 139.577,121.022 140.035,122.526 140.492,128.195 140.95,134.372 141.408,134.377 141.866,144.089 
 142.324,131.167 142.781,120.315 143.239,138.192 143.697,121.933 144.155,104.971 144.612,103.752 145.07,102.99 145.528,110.01 145.986,115.541 146.443,118.087 
 146.901,122.46 147.359,124.339 147.817,124.909 148.275,115.786 148.732,122.127 149.19,115.563 149.648,106.325 150.106,104.463 150.563,102.15 151.021,106.27 
 151.479,111.74 151.937,98.0095 152.395,77.4689 152.852,76.233 153.31,78.1627 153.768,87.9205 154.226,84.4581 154

It's easy to compare the simulated result with the deterministic steady state value implied from the theoretical model:

In [31]:
# mean of Labour
mean(sim_n[imax, :])

0.3307266542206806

In [32]:
# mean of Investment
mean(sim_i[imax, :])

0.20903002728386322

In [33]:
# mean of Capital
mean(sim_k[imax, :])

4.17107853941483

In [34]:
#set_calibration!(model,delta=0.05)
model.calibration[:controls]

2-element Array{Float64,1}:
 0.33   
 0.20961

In [35]:
model.calibration[:states]

2-element Array{Float64,1}:
 0.0    
 4.19221